# Code for calculating the dependence of the mean firing rate and coefficient of variation of the interspike intervals of the Brunel network on the synaptic strength J:

The following is a piece of code that uses the brian2 library to run the Brunel network for various values of J ranging from 0.01 to 0.06. We do not go beyond these values since significant correlations start to appear and the theory breaks down. The simulations are run for a total of 1000 ms for each J value.

**NOTE: THIS RETURNS THE STANDARD DEVIATION OF THE FIRING RATE NOT THAT OF THE INTERSPIKE INTERVALS**

In [ ]:
import random
from brian2 import *
import matplotlib.pyplot as plt
import numpy as np


def sim(g, nu_ext_over_nu_thr, sim_time, rate_tick_step, J):
    """
    g -- relative inhibitory to excitatory synaptic strength
    nu_ext_over_nu_thr -- ratio of external stimulus rate to threshold rate
    sim_time -- simulation time
    ax_spikes -- matplotlib axes to plot spikes on
    ax_rates -- matplotlib axes to plot rates on
    rate_tick_step -- step size for rate axis ticks
    """

    # network parameters
    N_E = 10000
    gamma = 0.25
    N_I = round(gamma * N_E)
    N = N_E + N_I
    epsilon = 0.005
    C_E = epsilon * N_E
    C_ext = C_E

    # neuron parameters
    tau = 20 * ms
    theta = 20 * mV
    V_r = 10 * mV
    tau_rp = 2 * ms

    # synapse parameters
    D = 1.5 * ms

    # external stimulus
    nu_thr = theta / (J * C_E * tau)

    defaultclock.dt = 0.1 * ms

    neurons = NeuronGroup(N,
                          """
                          dv/dt = -v/tau : volt (unless refractory)
                          """,
                          threshold="v > theta",
                          reset="v = V_r",
                          refractory=tau_rp,
                          method="exact",
    )

    excitatory_neurons = neurons[:N_E]
    inhibitory_neurons = neurons[N_E:]

    exc_synapses = Synapses(excitatory_neurons, target=neurons, on_pre="v += J", delay=D)
    exc_synapses.connect(p=epsilon)

    inhib_synapses = Synapses(inhibitory_neurons, target=neurons, on_pre="v += -g*J", delay=D)
    inhib_synapses.connect(p=epsilon)

    nu_ext = nu_ext_over_nu_thr * nu_thr

    external_poisson_input = PoissonInput(
        target=neurons, target_var="v", N=C_ext, rate=nu_ext, weight=J
    )

    rate_monitor = PopulationRateMonitor(neurons)
    
    run(sim_time, report='text')
    
    return np.mean(rate_monitor.rate/Hz),np.std(rate_monitor.rate/Hz)
        
    
params = {
        "g": 5,
        "nu_ext_over_nu_thr": 2,
        "t_range": [1000, 500],
        "rate_range": [0, 200],
        "rate_tick_step": 50,
}

Js = np.arange(0.01,0.6,0.01)*mV
means = np.zeros(Js.size)
stds = np.zeros(Js.size)

for i,j in enumerate(Js):
    print(i)
    print("J value: " + str(j))
    means[i],stds[i] = sim(params["g"],params["nu_ext_over_nu_thr"],1000* ms,params["rate_tick_step"],j)

plt.scatter(Js,means)
plt.scatter(Js,stds)


## The interspike intervals:

The following piece of code averages out the mean and standard deviation of the interspike intervals of the first 50 excitatory neurons in the Brunel network. Each simulation is run for 1000 ms. The values of J chosen are the same as the ones chosen above.

In [ ]:
import random
from brian2 import *
import matplotlib.pyplot as plt
import numpy as np


def sim(g, nu_ext_over_nu_thr, sim_time, rate_tick_step, J):
    """
    g -- relative inhibitory to excitatory synaptic strength
    nu_ext_over_nu_thr -- ratio of external stimulus rate to threshold rate
    sim_time -- simulation time
    ax_spikes -- matplotlib axes to plot spikes on
    ax_rates -- matplotlib axes to plot rates on
    rate_tick_step -- step size for rate axis ticks
    """

    # network parameters
    N_E = 10000
    gamma = 0.25
    N_I = round(gamma * N_E)
    N = N_E + N_I
    epsilon = 0.005
    C_E = epsilon * N_E
    C_ext = C_E

    # neuron parameters
    tau = 20 * ms
    theta = 20 * mV
    V_r = 10 * mV
    tau_rp = 2 * ms

    # synapse parameters
    D = 1.5 * ms

    # external stimulus
    nu_thr = theta / (J * C_E * tau)

    defaultclock.dt = 0.1 * ms

    neurons = NeuronGroup(N,
                          """
                          dv/dt = -v/tau : volt (unless refractory)
                          """,
                          threshold="v > theta",
                          reset="v = V_r",
                          refractory=tau_rp,
                          method="exact",
    )

    excitatory_neurons = neurons[:N_E]
    inhibitory_neurons = neurons[N_E:]

    exc_synapses = Synapses(excitatory_neurons, target=neurons, on_pre="v += J", delay=D)
    exc_synapses.connect(p=epsilon)

    inhib_synapses = Synapses(inhibitory_neurons, target=neurons, on_pre="v += -g*J", delay=D)
    inhib_synapses.connect(p=epsilon)

    nu_ext = nu_ext_over_nu_thr * nu_thr

    external_poisson_input = PoissonInput(
        target=neurons, target_var="v", N=C_ext, rate=nu_ext, weight=J
    )

    spike_monitor = SpikeMonitor(neurons[:50])
    
    run(sim_time, report='text')
    
    trains = spike_monitor.spike_trains()
    stdarray = np.zeros(50)
    meanarray = np.zeros(50)
    for i in range(50):
        if trains[i].size == 0:
            stdarray[i] = 0
            meanarray[i] = 0
        else:
            stdarray[i] = np.std(np.diff(trains[i]))
            meanarray[i] = np.mean(np.diff(trains[i]))
    
    return np.mean(stdarray),np.mean(meanarray)
    

Js = np.arange(0.01,0.6,0.01)*mV
intervalstds = np.zeros(Js.size)
intervalmeans = np.zeros(Js.size)

for i,j in enumerate(Js):
    print(i)
    print("J value: " + str(j))
    intervalstds[i],intervalmeans[i] = sim(params["g"],params["nu_ext_over_nu_thr"],1000* ms,params["rate_tick_step"],j)
    print(intervalstds)
    print(intervalmeans)

In [ ]:
intervalcvs = intervalstds/intervalmeans
intervalcvs.size

In [ ]:
plt.scatter(Js,intervalcvs)

np.savetxt("intervalcvs",intervalcvs)